In [1]:
pip install geopy pandas


  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
###Geocoding using geopy

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Initialize the geocoder with Nominatim
geolocator = Nominatim(user_agent="myGeocoder")

# Set up rate limiter to avoid being blocked by the server (2 seconds delay)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

# Load your dataset (assumes a CSV file with a column named 'Address')
data = pd.read_csv("/Users/supriya/Downloads/Banking Export Data CSV-5.csv")

# Function to apply geocoding
def geocode_address(Address):
    try:
        location = geocode(Address)
        return (location.latitude, location.longitude) if location else (None, None)
    except Exception as e:
        return (None, None)

# Apply the geocoding function to the 'address' column
data['coordinates'] = data['Address'].apply(geocode_address)

# Split coordinates into 'latitude' and 'longitude' columns
data[['latitude', 'longitude']] = pd.DataFrame(data['coordinates'].tolist(), index=data.index)

# Drop the 'coordinates' column if you don't need it anymore
data.drop(columns=['coordinates'], inplace=True)

# Save the geocoded data to a new CSV file
data.to_csv("geocoded_addresses.csv", index=False)

print("Geocoding complete!")

##completly broke -- couldn't get a result

In [3]:
!pip install opencage


  Obtaining dependency information for opencage from https://files.pythonhosted.org/packages/8e/91/012b9f0561a56aef20a748ca4ac18358165aff02e5ce462a9a3a7c4fda67/opencage-3.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for backoff>=2.2.1 from https://files.pythonhosted.org/packages/df/73/b6e24bd22e6720ca8ee9a85a0c4a2971af8497d8f3193fa05390cbd46e09/backoff-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for tqdm>=4.66.4 from https://files.pythonhosted.org/packages/48/5d/acf5905c36149bbaec41ccf7f2b68814647347b72075ac0b1fe3022fdc73/tqdm-4.66.5-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
  Obtaining dependency information for certifi>=2024.07.04 from https://files.pythonhosted.org/packages/12/90/3c9ff0512038035f59d279fddeb79f5f1eccd8859f06d6163c58798b9487/certifi-2024.8.30-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp>=3.10.5 from https://files.pythonhosted.org/packa

In [ ]:
###With opencage API

import pandas as pd
from opencage.geocoder import OpenCageGeocode
import time
import os

# Replace 'your_api_key' with your actual OpenCage API key
API_KEY = '9235c10eeeb945b69d0b979d0ba8bd63'
geocoder = OpenCageGeocode(API_KEY)

# Function to geocode an address using OpenCage
def geocode_Address(Address):
    try:
        result = geocoder.geocode(Address, no_annotations='1')
        if result and len(result):
            # Extract the first result's latitude and longitude
            return result[0]['geometry']['lat'], result[0]['geometry']['lng']
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {Address}: {e}")
        return None, None

# Load dataset
input_file = '/Users/supriya/Downloads/Banking Export Data CSV-5.csv'
output_file = '/Users/Supriya/Downloads/geocoded_addresses.csv'

# Check if the output file already exists (if the process was previously started)
if os.path.exists(output_file):
    # Load the existing geocoded data
    df = pd.read_csv(output_file)
    print(f"Resuming from existing file {output_file}...")
else:
    # If the file doesn't exist, load the original dataset and create new columns
    df = pd.read_csv(input_file)
    df['latitude'] = None
    df['longitude'] = None

# Geocode each address that hasn't been geocoded yet
for index, row in df.iterrows():
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):
        Address = row['Address']
        lat, lng = geocode_Address(Address)

        # Store the results in the dataframe
        df.at[index, 'latitude'] = lat
        df.at[index, 'longitude'] = lng

        # Print progress
        print(f"Geocoded {Address}: {lat}, {lng}")

        # Save after each iteration to avoid data loss
        df.to_csv(output_file, index=False)

        # Respect the rate limit (typically 1 request per second)
        time.sleep(1)  # 1-second delay

print("Geocoding complete. Data saved to geocoded_addresses.csv")


###Slower and higher Nulls (~50%). 2500 entries are free


Geocoded SHAHAPUR ROAD AT SHIRISHPADA PO KONE TAL WADA DIST PALGHAR (THANE)SHAHAPUR ROAD AT SHIRISHPADA PO KONE TAL WADA DIST PALGHAR (THANE),-1,PALGHAR,MAHARASHTRA,421303: None, None
Geocoded SONAR ALI SAMBHAJI PATH AT POST JAWHAR (RURAL)THANE 27SONAR ALI SAMBHAJI PATH AT POST JAWHAR (RURAL)THANE 27,-1,PALGHAR,MAHARASHTRA,401603: None, None
Geocoded SHOP NO 6 CHNCH PADAAAZAD NAGAR GOKHIVRE VASAI EASTSHOP NO 6 CHNCH PADAAAZAD NAGAR GOKHIVRE VASAI EAST7,-1,PALGHAR,MAHARASHTRA,401208: None, None
Geocoded 203 GODAVARI APT JAVHAR RD NR 3RD PETROL PUMP RAMWADI DAHANU ROAD E THANE 27203 GODAVARI APT JAVHAR RD NR 3RD PETROL PUMP RAMWADI DAHANU ROAD E THANE 27,-1,PALGHAR,MAHARASHTRA,401406: None, None
Geocoded AT PIMPALGAON AT PIMPALGAON 27AT PIMPALGAON AT PIMPALGAON 27,-1,PALGHAR,MAHARASHTRA,401603: 47.27541, 8.4897
Geocoded AT POST AKARE JAWHAR AT POST AKARE JAWHAR 27AT POST AKARE JAWHAR AT POST AKARE JAWHAR 27,-1,PALGHAR,MAHARASHTRA,401603: 33.19122, -101.379
Geocoded KHARONDA JAWHAR KHARON